In [1]:
import time
import pickle
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from newsapi import NewsApiClient
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from apscheduler.schedulers.background import BackgroundScheduler

In [2]:
pip install apscheduler 

Note: you may need to restart the kernel to use updated packages.


In [3]:
# API Key for NewsAPI
API_KEY = 'bcae1bfec35f4e78a3cdc7accdbffd86'
newsapi = NewsApiClient(api_key=API_KEY)


In [5]:
# Load or initialize model and vectorizer
try:
    with open("sentiment_model.pkl", "rb") as model_file:
        model = pickle.load(model_file)
    with open("vectorizer.pkl", "rb") as vec_file:
        vectorizer = pickle.load(vec_file)
    print("Model and vectorizer loaded successfully.")
except FileNotFoundError:
    print("No existing model found. Training from scratch.")
    model = LogisticRegression(max_iter=500)
    vectorizer = TfidfVectorizer(max_features=5000)

Model and vectorizer loaded successfully.


In [7]:
# Fetch real-time news
def fetch_real_time_news():
    try:
        articles = newsapi.get_everything(q='stock market', language='en', sort_by='publishedAt')
        news_data = [{
            'publishedAt': article['publishedAt'],
            'text': f"{article['title']} {article['description']} {article['content']}"
        } for article in articles['articles']]
        return pd.DataFrame(news_data)
    except Exception as e:
        print(f"Error fetching news: {e}")
        return pd.DataFrame()

In [9]:
# Text preprocessing
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [11]:
# Retrain model
def retrain_model(new_data):
    global model, vectorizer
    try:
        labeled_data = pd.read_csv("labeled_news_data.csv")
    except FileNotFoundError:
        labeled_data = pd.DataFrame(columns=["clean_text", "sentiment"])
    
    labeled_data = pd.concat([labeled_data, new_data], ignore_index=True)
    
    X = labeled_data['clean_text']
    y = labeled_data['sentiment']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    model.fit(X_train_vec, y_train)
    
    with open("sentiment_model.pkl", "wb") as model_file:
        pickle.dump(model, model_file)
    with open("vectorizer.pkl", "wb") as vec_file:
        pickle.dump(vectorizer, vec_file)
    
    print("Model retrained and updated successfully!")
    evaluate_model(X_test_vec, y_test)


In [13]:
# Evaluate model
def evaluate_model(X_test_vec, y_test):
    y_pred = model.predict(X_test_vec)
    print("\nModel Accuracy:", accuracy_score(y_test, y_pred))
    print("\nClassification Report:\n", classification_report(y_test, y_pred))
    
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix")
    plt.show()

In [15]:
# Automated training pipeline
def automate_training():
    print("Fetching and processing new news data...")
    real_time_data = fetch_real_time_news()
    if not real_time_data.empty:
        real_time_data['clean_text'] = real_time_data['text'].apply(clean_text)
        real_time_data['sentiment'] = model.predict(vectorizer.transform(real_time_data['clean_text']))
        real_time_data[['clean_text', 'sentiment']].to_csv("labeled_news_data.csv", mode='a', header=False, index=False)
        retrain_model(real_time_data[['clean_text', 'sentiment']])
    else:
        print("No new data fetched.")

In [ ]:
# Scheduler to run training every hour
scheduler = BackgroundScheduler()
scheduler.add_job(automate_training, 'interval', hours=1)
scheduler.start()

print("Automated model training started. Running every hour...")
try:
    while True:
        time.sleep(60)
except (KeyboardInterrupt, SystemExit):
    scheduler.shutdown()
    print("Scheduler stopped.")

Automated model training started. Running every hour...
Fetching and processing new news data...


Job "automate_training (trigger: interval[1:00:00], next run at: 2025-02-10 20:02:53 IST)" raised an exception
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/apscheduler/executors/base.py", line 131, in run_job
    retval = job.func(*job.args, **job.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/wp/l1lbtpb57rl387801fcn64ch0000gn/T/ipykernel_9160/3781866117.py", line 9, in automate_training
    retrain_model(real_time_data[['clean_text', 'sentiment']])
  File "/var/folders/wp/l1lbtpb57rl387801fcn64ch0000gn/T/ipykernel_9160/2292364875.py", line 5, in retrain_model
    labeled_data = pd.read_csv("labeled_news_data.csv")
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/pandas

In [ ]:

# Load labeled dataset
file_path = "labeled_news_data.csv"
df = pd.read_csv(file_path, names=["publishedAt", "clean_text", "sentiment"])  # Ensure correct column names



In [ ]:
# Drop missing values
df = df.dropna()

# Vectorize text data
with open("vectorizer.pkl", "rb") as vec_file:
    vectorizer = pickle.load(vec_file)

X = vectorizer.transform(df['clean_text'])  # Transform existing data
y = df['sentiment']  # Labels (-1 = Negative, 0 = Neutral, 1 = Positive)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the trained model
with open("sentiment_model.pkl", "rb") as model_file:
    model = pickle.load(model_file)

# Make predictions
y_pred = model.predict(X_test)

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print Accuracy and Classification Report
print("\n Model Accuracy:", accuracy)
print("\n Classification Report:\n", classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)


In [ ]:


plt.figure(figsize=(5,5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Negative", "Neutral", "Positive"], yticklabels=["Negative", "Neutral", "Positive"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()
